In [1]:
import sys
print(sys.executable)
print(sys.path)
print(sys.version)

/home2/s418610/.conda/envs/py37_res_diffxpy/bin/python
['/home2/s418610/jupyter_notebooks/03.Invivo_neuronal_reprogramming/Manuscript/to_GitHub', '', '/cm/shared/apps/cnvkit/0.9.5/lib/python3.6/site-packages', '/cm/shared/apps/python/3.6.4-anaconda/lib', '/cm/shared/apps/jupyterkernel-matlab/0.16.1/lib/python3.6/site-packages', '/cm/shared/apps/python/2.7.x-anaconda/lib', '/home2/s418610/.conda/envs/py37_res_diffxpy/lib/python37.zip', '/home2/s418610/.conda/envs/py37_res_diffxpy/lib/python3.7', '/home2/s418610/.conda/envs/py37_res_diffxpy/lib/python3.7/lib-dynload', '/home2/s418610/.local/lib/python3.7/site-packages', '/home2/s418610/.conda/envs/py37_res_diffxpy/lib/python3.7/site-packages', '/home2/s418610/.conda/envs/py37_res_diffxpy/lib/python3.7/site-packages/fa2-0.3.5-py3.7-linux-x86_64.egg', '/home2/s418610/.conda/envs/py37_res_diffxpy/lib/python3.7/site-packages/batchglm', '/home2/s418610/.conda/envs/py37_res_diffxpy/lib/python3.7/site-packages/diffxpy', '/home2/s418610/.conda/e

In [2]:
import re
from pathlib import Path
import numpy as np
import pandas as pd
import scipy.sparse
import sklearn.manifold
import statsmodels.api as sm

# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from pyclustering.cluster.kmedoids import kmedoids

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# import seaborn as sb

import scanpy as sc
import harmonypy as hm

/home2/s418610/.conda/envs/py37_res_diffxpy/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home2/s418610/.conda/envs/py37_res_diffxpy/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1

In [3]:
plt.rcParams['figure.figsize']=(8,8) #rescale figures
sc.settings.verbosity = 3

print('numpy', np.__version__)
print('pandas', pd.__version__)
print('scipy', scipy.__version__)
print('sklearn', sklearn.__version__)
print('statsmodels', sm.__version__)
print('matplotlib', mpl.__version__)
print('seaborn', sns.__version__)

sc.logging.print_versions()

!date +%F

numpy 1.20.3
pandas 0.25.2
scipy 1.4.1
sklearn 0.21.3
statsmodels 0.10.1
matplotlib 3.1.3
seaborn 0.9.0
scanpy==1.4.5.post1 anndata==0.6.22.post1 umap==0.3.10 numpy==1.20.3 scipy==1.4.1 pandas==0.25.2 scikit-learn==0.21.3 statsmodels==0.10.1 python-igraph==0.7.1 louvain==0.6.1
2021-08-23


In [4]:
import anndata
import pickle

In [5]:
import statsmodels.api as sm
lowess = sm.nonparametric.lowess

In [6]:
from scipy.stats import chi2_contingency

In [7]:
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

In [8]:
import os

# set a working directory for saving plots
wdir = "/project/GCRB/Hon_lab/s418610/Projects/03.Invivo_neuronal_reprogramming/99.Manuscript/Figures_revision/"
os.chdir( wdir )

In [9]:
data_dir = '/project/GCRB/Hon_lab/s418610/Projects/03.Invivo_neuronal_reprogramming/analysis/Cluster_with_10X_20k/data/10x_remapped/Revision/'

In [10]:
neural_data_dir = data_dir + 'NeuralOnly/'

In [11]:
geneclustering_dir = data_dir + 'GeneClustering/'

## Table S1: Gene symbols in gene clusters

In [12]:
input_path = geneclustering_dir + 'adata_genes'
with open(input_path, 'rb') as input_file:
 
    adata_genes = pickle.load(input_file)

In [13]:
adata_genes

AnnData object with n_obs × n_vars = 2584 × 4607 
    obs: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'p_val_adj', 'n_counts', 'n_cells_experimental', 'louvain', 'log10_n_cells_experimental'
    var: 'library_name', 'batch', 'lenti_count', 'lenti_log1p_count', 'tdT_count', 'tdT_log1p_count', 'doublet_scores', 'predicted_doublets', 'n_counts', 'log_counts', 'n_genes', 'mt_count', 'mt_frac', 'n_counts_all', 'louvain', 'dpt_pseudotime', 'rank_pseudotime'
    uns: 'pca', 'neighbors', 'louvain', 'paga', 'louvain_sizes', 'louvain_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'

In [36]:
with pd.ExcelWriter('./tables/GeneClusters.xlsx') as writer:  
    for current_cluster in adata_genes.obs['louvain'].cat.categories:         
        current_genes = adata_genes.obs.loc[adata_genes.obs['louvain'] == current_cluster, :].index
        current_genes = pd.DataFrame(data=current_genes, columns=['Gene Symbol'])
        
        current_genes.to_excel(writer, sheet_name='GeneCluster'+current_cluster, header=True, index=False)

## Table S2: GOEA analysis results for gene clusters

In [55]:
GOEA_dir = '/project/GCRB/Hon_lab/s418610/Projects/03.Invivo_neuronal_reprogramming/analysis/Cluster_with_10X_20k/data/10x_remapped/Revision/GOEA/'

In [56]:
input_path = GOEA_dir + 'goea_DF_gene_clusters'
with open(input_path, 'rb') as input_file:
 
    goea_DF_gene_clusters = pickle.load(input_file)

In [57]:
assert len(goea_DF_gene_clusters)==12

In [62]:
with pd.ExcelWriter('./tables/GeneClusterGOEA.xlsx') as writer:  

    for cluster, goea_summary_MF, goea_summary_BP, goea_summary_CC in goea_DF_gene_clusters:

        goea_summary_BP.to_excel(writer, sheet_name='GeneCluster'+cluster, header=True, index=False)

## Table S3: Enriched genes in WT E18.5 neurogenesis vs DLX2-induced reprogramming

In [17]:
from goatools.test_data.genes_NCBI_10090_ProteinCoding import GENEID2NT as GeneID2nt_mus
# from goatools.test_data.genes_NCBI_10090_All import GENEID2NT as GeneID2nt_mus

In [18]:
len(GeneID2nt_mus.keys())

28212

In [19]:
GeneID2Symbol_dict = {}

In [20]:
for GeneID in list(GeneID2nt_mus.keys()):
    Symbols = []
    Symbols.append(GeneID2nt_mus[GeneID].Symbol)
    if not GeneID2nt_mus[GeneID].Aliases == '':
        for alias in GeneID2nt_mus[GeneID].Aliases.split(','):
            Symbols.append(alias.strip())
    GeneID2Symbol_dict[GeneID] = Symbols

In [12]:
DE_dir = '/project/GCRB/Hon_lab/s418610/Projects/03.Invivo_neuronal_reprogramming/analysis/Cluster_with_10X_20k/data/10x_remapped/Revision/DE_analysis/'

In [13]:
input_path = DE_dir + 'embryonic_neurogenesis_test_list_collection_100iter'
with open(input_path, 'rb') as input_file:
 
    test_list_collection = pickle.load(input_file)

In [14]:
assert len(test_list_collection) == 100

In [42]:
cell_clusters = [cluster for cluster, _ in test_list_collection[0]]

In [49]:
df_experiment_enriched_genes = pd.DataFrame(index=range(100), columns=cell_clusters, dtype=object)
df_public_enriched_genes = pd.DataFrame(index=range(100), columns=cell_clusters, dtype=object)


for iteration, test_list_all in enumerate(test_list_collection): 

    for cluster, test_cluster in test_list_all:
        experiment_up_genes = test_cluster.summary()[np.logical_and(test_cluster.summary()['qval'] < 1e-2, test_cluster.summary()['log2fc'] > 1)]
        experiment_up_genes = experiment_up_genes['gene'].values

        df_experiment_enriched_genes.loc[iteration, cluster] = experiment_up_genes

                    
    for cluster, test_cluster in test_list_all:
        public_up_genes = test_cluster.summary()[np.logical_and(test_cluster.summary()['qval'] < 1e-2, test_cluster.summary()['log2fc'] < -1)]
        public_up_genes = public_up_genes['gene'].values


        df_public_enriched_genes.loc[iteration, cluster] = public_up_genes



In [50]:
df_experiment_enriched_genes.index.name = 'interation'
df_public_enriched_genes.index.name = 'interation'

In [51]:
with pd.ExcelWriter('./tables/EmbryonicNeurogenesis_Reprogramming_DEGs.xlsx') as writer:  

    df_experiment_enriched_genes.to_excel(writer, sheet_name='Reprogramming_enriched', header=True, index=True)
    df_public_enriched_genes.to_excel(writer, sheet_name='E18.5Neurogenesis_enriched', header=True, index=True)